# This script aims at developing the build_shapes script on the basis of PyPSA-Eur

In [12]:
# import
import os
import sys

# IMPORTANT: RUN SCRIPT FROM THIS SCRIPTS DIRECTORY i.e data_exploration/ TODO: make more robust
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append("./../../scripts")

import logging
import shutil

import geopandas as gpd
import geoplot
import fiona

import numpy as np
import pandas as pd
import requests
import zipfile
import matplotlib.pyplot as plt
from esy.osmfilter import run_filter
from esy.osmfilter import Node, Relation, Way
from esy.osmfilter import osm_info as osm_info
from esy.osmfilter import osm_pickle as osm_pickle

from iso_country_codes import AFRICA_CC
#from ..scripts.iso_country_codes import AFRICA_CC

from shapely.geometry import LineString, Point, Polygon

logger = logging.getLogger(__name__)

In [13]:
test_CC = {"ZAF": "SOUTH AFRICA", "NGA": "Nigeria"} # or any other country
update = True

def download_GADM(country_code, update=False):
    """
    Download gpkg file from GADM for a given country code

    Parameters
    ----------
    country_code : str 
        Three letter country codes of the downloaded files 
    update : bool 
        Name of the network component 
        Update = true, forces re-download of files

    Returns
    -------
    gpkg file per country

    """

    GADM_filename = f"gadm36_{country_code}"
    GADM_url = f"https://biogeo.ucdavis.edu/data/gadm3.6/gpkg/{GADM_filename}_gpkg.zip"

    GADM_inputfile_zip = os.path.join(
        os.getcwd(), "data", "gadm", GADM_filename, GADM_filename + ".zip"
    )  # Input filepath zip

    GADM_inputfile_gpkg = os.path.join(
        os.getcwd(), "data", "gadm", GADM_filename, GADM_filename + ".gpkg"
    )  # Input filepath gpkg

    if not os.path.exists(GADM_inputfile_zip) or update is True:
        print(f"{GADM_filename} does not exist, downloading to {GADM_inputfile_zip}")
        #  create data/osm directory
        os.makedirs(os.path.dirname(GADM_inputfile_zip), exist_ok=True)

        with requests.get(GADM_url, stream=True) as r:
            with open(GADM_inputfile_zip, "wb") as f:
                shutil.copyfileobj(r.raw, f)

        with zipfile.ZipFile(GADM_inputfile_zip,"r") as zip_ref:
            zip_ref.extractall(os.path.dirname(GADM_inputfile_zip))
    
    return GADM_inputfile_gpkg, GADM_filename

# download GADM gpkg files of selected countries
for country_code in test_CC.keys():
    # download file gpkgS
    file_gpkg, _ = download_GADM(country_code, False)
    print(file_gpkg)

print("Completed")

c:\Users\Davide\Git\pypsa-africa\data_exploration\WP5_transmission_assets\data\gadm\gadm36_ZAF\gadm36_ZAF.gpkg
c:\Users\Davide\Git\pypsa-africa\data_exploration\WP5_transmission_assets\data\gadm\gadm36_NGA\gadm36_NGA.gpkg
Completed


In [14]:
# download gpkg files of selected countries and append them to a single GeoDataFrame for each desired layer

layers = [0, 2]

# initialize dictionary for all geodataframe by layer
geodf_all = {layer: gpd.GeoDataFrame() for layer in layers}

def get_GADM_layer(country_list, layer, update=False):
    # initialization of the geoDataFrame
    geodf_GADM = gpd.GeoDataFrame()

    for country_code in country_list:
        # download file gpkg
        file_gpkg, name_file = download_GADM(country_code, False)

        # read gpkg file
        geodf_temp = gpd.read_file(file_gpkg, layer=name_file+"_"+str(layer))

        # append geodataframes
        geodf_GADM = geodf_GADM.append(geodf_temp)
    
    geodf_GADM.reset_index(drop=True, inplace=True)

    return geodf_GADM

geodf_GADM_countries = get_GADM_layer(test_CC.keys(), 0, update=False) # layer 0 is country shapes
geodf_GADM_NUTS3 = get_GADM_layer(test_CC.keys(), 2, update=False) # layer 2 are the most fine shapes available (to be checked if always available)

In [15]:
geodf_GADM_countries

,GID_0,NAME_0,geometry
0,ZAF,South Africa,"MULTIPOLYGON (((19.42097 -34.67903, 19.42097 -..."
1,NGA,Nigeria,"MULTIPOLYGON (((6.44042 4.30875, 6.44042 4.307..."


In [16]:
#geodf_all[1].plot()

In [17]:
# # download file gpkg
# file_gpkg = download_gpkg(list(test_CC.keys())[0], False)

# print(fiona.listlayers(file_gpkg))

# # read gpkg file
# geodf_temp = gpd.read_file(file_gpkg, layer="gadm36_ZAF_0")
# geodf_temp.plot()

In [18]:
def load_EEZ(selected_countries_codes, name_file = "eez_v11.gpkg"):
    EEZ_gpkg = os.path.join(
        os.getcwd(), "data", "raw", "eez", name_file
    )  # Input filepath gpkg

    if not os.path.exists(EEZ_gpkg):
        raise Exception(f"File EEZ {name_file} not found, please download it from https://www.marineregions.org/download_file.php?name=World_EEZ_v11_20191118_gpkg.zip and copy it in {os.path.dirname(EEZ_gpkg)}")
    
    geodf_EEZ = gpd.read_file(EEZ_gpkg)
    geodf_EEZ = geodf_EEZ[["ISO_TER1", "geometry"]] # [["ISO_TER1", "TERRITORY1", "ISO_SOV1", "ISO_SOV2", "ISO_SOV3", "geometry"]]
    geodf_EEZ = geodf_EEZ[[any([x in selected_countries_codes]) for x in geodf_EEZ["ISO_TER1"]]]
    geodf_EEZ.reset_index(drop=True, inplace=True)
    
    geodf_EEZ.rename(columns = {"ISO_TER1": "name"}, inplace=True)

    return geodf_EEZ

geodf_EEZ = load_EEZ(test_CC.keys())

#print(geodf_EEZ.keys())
geodf_EEZ.head()

,name,geometry
0,ZAF,"MULTIPOLYGON (((36.53069 -27.71605, 36.53058 -..."
1,NGA,"MULTIPOLYGON (((8.65029 4.93224, 8.65051 4.932..."


In [19]:
file_country_shapes_clean = os.path.join(os.getcwd(), "data", "clean", "eez", "country_shapes" + ".geojson")
file_off_shore_shapes_clean = os.path.join(os.getcwd(), "data", "clean", "eez", "off_shore_shapes" + ".geojson")
os.makedirs(os.path.dirname(file_country_shapes_clean), exist_ok=True)

# country shapes
geodf_country = geodf_GADM_countries[["GID_0", "geometry"]].copy()
geodf_country.rename(columns = {"GID_0": "name"}, inplace=True)

geodf_country.to_file(file_country_shapes_clean, driver="GeoJSON")  # Generate GeoJson
geodf_EEZ.to_file(file_off_shore_shapes_clean, driver="GeoJSON")  # Generate GeoJson

ERROR:fiona._env:c:\Users\Davide\Git\pypsa-africa\data_exploration\WP5_transmission_assets\data\clean\eez\country_shapes.geojson: No such file or directory
ERROR:fiona._env:c:\Users\Davide\Git\pypsa-africa\data_exploration\WP5_transmission_assets\data\clean\eez\off_shore_shapes.geojson: No such file or directory
